In [2]:
#MAKING THE BRAIN TUMOR MODEL WITH INCEPTION V3 PRETRAINED MODEL

#Importing libraries
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D
from tensorflow.keras import Model
import PIL
#Pretrained model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#FEEDING DATA INTO THE MODEL
from tensorflow.keras.preprocessing.image import ImageDataGenerator #allows us to augment and process data fed into a CNN

In [4]:
path_inception = "D:/bcd/data/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
local_weights_file = path_inception
pre_trained_model = InceptionV3(input_shape = (240, 240, 3), 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights(local_weights_file)

In [5]:
#Locking the weights and parameters of pretrained model
for layer in pre_trained_model.layers:
    layer.trainable = False

#Getting desired layer output
last_layer = pre_trained_model.get_layer('mixed8')
last = last_layer.output
print('Last layer output shape : ', last.shape)

Last layer output shape :  (?, 6, 6, 1280)


In [6]:
from tensorflow.keras.losses import categorical_crossentropy
# Model configuration
batch_size = 50
img_size, img_num_channels = 240, 3
loss_function = categorical_crossentropy
no_classes = 4
no_epochs = 15
optimizer = Adam()
verbosity = 1

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import os,cv2,random
import numpy as np
import pandas as pd

In [8]:
data_path='D:/bcd/data/trainingImages'
list_folder=os.listdir(path = data_path)
data=[]
im_size=240    
for i in list_folder:
    new_path=os.path.join(data_path,i) 
    pic_list=os.listdir(new_path)                                               
    for img in pic_list:
        pic=os.path.join(new_path,img)   
        arr=cv2.imread(pic)    
        data.append([arr,list_folder.index(i)])    
        
random.shuffle(data)  
x_train,y_train=[],[]
for i,j in data:
    x_train.append(i)
    y_train.append(j)
x_train=np.array(x_train).reshape(-1,im_size,im_size,3)
y_train=np.array(y_train).reshape(-1,1)

encoder = OneHotEncoder()
y_train = encoder.fit_transform(y_train)
X_train = x_train/255
y_train = y_train.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X_train,y_train, test_size=0.25)

In [9]:
# Parse numbers as floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [11]:
# Define per-fold score containers <-- these are new
acc_per_fold = [] #stores accuracy
loss_per_fold = [] #stores losses

In [12]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((Y_train, Y_test), axis=0)

In [13]:
#FEEDING DATA INTO THE MODEL
from tensorflow.keras.preprocessing.image import ImageDataGenerator #A tool that allows us to augment and process data to be fed into a CNN

#class for training set
train_data = ImageDataGenerator(rescale = 1./255, #pixel values rescaled so that it stays between 0 and 1.
                                shear_range = 0.2, #this and the next 4 steps are used to augment our data by shearing it, flipping and zooming it to produce more examples per image.
                                zoom_range = 0.2,  #Data augmentation allows us to prevent overfitting of data into the training set.
                                horizontal_flip = True,
                                vertical_flip = True,
                                rotation_range = 40,
                                width_shift_range = 0.2,
                                height_shift_range = 0.2,
                                fill_mode = 'nearest'
                                )
test_data = ImageDataGenerator(rescale = 1/.255)

train_set = train_data.flow_from_directory('D:/bcd/data/trainingImages', #Image path
                                           target_size = (240,240), #The shape which we want to input our images in our model
                                           batch_size = 20,         #The batch size in mini batch gradient descent
                                           class_mode = 'categorical')   


test_set = test_data.flow_from_directory('D:/bcd/data/validationImages',
                                         target_size = (240,240),
                                         batch_size = 50,
                                         class_mode = 'categorical')

Found 962 images belonging to 4 classes.
Found 500 images belonging to 4 classes.


In [14]:
from sklearn.model_selection import KFold
import numpy as np

In [15]:
num_folds=10

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1

In [17]:
for train, test in kfold.split(inputs, targets):
    #Building own model on top of trained network
    x = Conv2D(500, (1,1), activation = 'relu')(last)
    x = Flatten()(x)
    x = Dense(1028, activation = 'relu')(x)
    x = Dropout(rate = 0.1)(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dropout(rate = 0.1)(x)
    x = Dense(4, activation = 'softmax')(x)

    #Compiling model
    model = Model(inputs = pre_trained_model.input, outputs = x, name = 'Predict')
    opt1 = Adam(learning_rate = 0.001)
    opt2 = RMSprop(learning_rate = 0.001)
    model.compile(optimizer = opt1 , loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity,
              #validation_data = test_set,
              #validation_steps = 5          
              )
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/15
865/865 [==============================] - 262s 303ms/sample - loss: 3.5437 - acc: 0.2983
Epoch 2/15
865/865 [==============================] - 306s 354ms/sample - loss: 0.7271 - acc: 0.7156
Epoch 3/15
865/865 [==============================] - 305s 352ms/sample - loss: 0.3221 - acc: 0.8936
Epoch 4/15
865/865 [==============================] - 299s 346ms/sample - loss: 0.1631 - acc: 0.9376
Epoch 5/15
865/865 [==============================] - 313s 361ms/sample - loss: 0.0878 - acc: 0.9723
Epoch 6/15
865/865 [==============================] - 263s 304ms/sample - loss: 0.0392 - acc: 0.9861
Epoch 7/15
865/865 [==============================] - 292s 338ms/sample - loss: 0.0154 - acc: 0.9954
Epoch 8/15
865/865 [==============================] - 295s 341ms/sample - loss: 0.0218 - acc: 0.9919
Epoch 9/15
865/865 [==============================] - 290s 335ms/sample - loss: 0.0177 - acc: 0

In [19]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.5871885579425035 - Accuracy: 81.44329786300659%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.38549215523202407 - Accuracy: 90.7216489315033%
------------------------------------------------------------------------
> Fold 3 - Loss: 3.5370729764302573 - Accuracy: 61.45833134651184%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.2696586698293686 - Accuracy: 91.66666865348816%
------------------------------------------------------------------------
> Fold 5 - Loss: 1.6566379070281982 - Accuracy: 61.45833134651184%
------------------------------------------------------------------------
> Fold 6 - Loss: 1.581248164176941 - Accuracy: 67.70833134651184%
------------------------------------------------------------------------
